In [1]:
import os
os.chdir('../')

In [2]:
from dataclasses import dataclass
from pathlib import Path
from src.constants import *
from src.utils.utlis import *
from src.exceptions.expection import CustomException
import pandas as pd
from sklearn.model_selection import train_test_split
import sys


@dataclass(frozen=True)

class DataIngestionConfig:
    root_dir: Path
    local_data_file: Path
    train_file_path:Path
    test_file_path:Path

In [3]:
class ConfigManager:
    def __init__(self,config_file=CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config=read_yaml(config_file)
        self.params=read_yaml(params_filepath)
    

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            local_data_file= config.local_data_file,
            train_file_path=config.train_file_path,
            test_file_path=config.test_file_path

        )    
        return data_ingestion_config

In [4]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config

    def initate_data_ingestion(self):
        logger.info('Data Ingestion Started')
        try:
            df=pd.read_csv('data/tested.csv')


            train_set,test_set=train_test_split(df,test_size=.2,random_state=42)

            train_set.to_csv(self.config.train_file_path,index=False,header=True)

            test_set.to_csv(self.config.test_file_path,index=False,header=True)

            logger.info('Data Ingestion finished')

            # return (
            #     self.config.train_file_path,
            #     self.config.test_file_path
            # )
        except Exception as e:
            raise CustomException(e,sys)  

In [5]:
try:
    config=ConfigManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(data_ingestion_config)
    data_ingestion.initate_data_ingestion()
except Exception as e:
    raise CustomException(e,sys)